In [85]:
example_topics_list = ['animals', 'linters', 'polkadot', 'planetscale', 'prometheus', 'rocksdb', 'design-systems', 'figma-plugins',
                        'fonts', 'load-balancing', 'finance', 'solar', 'veganism', 'exercises', 'fitness', 'running', 'strength-training', 
                        'yoga', 'rules', 'recipes', '2022-july', 'movies', 'music-artists', 'massages', 'logic-pro', 'caddy', 'decentralization', 
                        'puzzles', 'web-presence', 'quantum-computing', 'neo4j', 'parsing', 'python-libraries', 'a-view-of-mathematics', 'drones', 
                        'robots', 'cryptography', 'encryption', 'blogs', 'physics', 'dat', 'my-file-system', 'safari', 'spin', '3d-printing', 
                        'consciousness', 'ego', 'finding-work', 'freelancing', 'research', 'ambient', 'analytics', 'grafana', 'tinybird', 'birds', 
                        'anki', 'api', 'trpc', 'anime', 'architecture', 'art', 'arkit', 'augmented-reality', 'automation', 'home-automation', 
                        'backups', 'bioinformatics', 'documentaries', 'cannabis', 'clothes', 'comics', 'dancing', 'drawing', 'furniture', 
                        'generative-art', 'makeup', 'midjourney', 'pen-plotting', 'photography', 'sketching', 'tattoos', 'cracking-the-coding-interview', 
                        'restaurants', 'articles', 'biology', 'bionics', 'computational-biology', 'evolution', 'dna', 'genomics', 'immunology', 
                        'immunotherapy', 'regenerative-medicine', 'viruses', 'ai-modern-approach', 'books', 'brave-new-world', 'code-the-hidden-language', 
                        'arweave', 'blockchain']

In [86]:
# !pip3 install sentence_transformers
# !pip3 install faiss-cpu

#### 1. Saving current topics embeddings

In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('TaylorAI/gte-tiny')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/669 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
embeddings = model.encode(example_topics_list)
embeddings_dict = {}
for i in range(embeddings.shape[0]):
    embeddings_dict[i] = example_topics_list[i]

In [21]:
import faiss
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

#### 2. Function to get nearest topics to query

In [38]:
def get_candidates(query, k=3):
    query_em = model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_em, k)
    for i in indices[0]:
        print(embeddings_dict[i])

In [39]:
get_candidates('Tensorflow')

analytics
computational-biology
ai-modern-approach


In [40]:
get_candidates('Python')

python-libraries
api
computational-biology


In [41]:
get_candidates('Programming AI wife')

ai-modern-approach
automation
home-automation


#### 3. Function to get info about topic from query

In [79]:
import requests
from bs4 import BeautifulSoup
import re 
from urllib.parse import urlparse

def clean_url(url):
    parsed_url = urlparse(url)
    path = parsed_url.path
    netloc = parsed_url.netloc
    combined_string = f"{netloc}{path}"
    cleaned_string = re.sub(r'www|https?|::|//|-|\.(com|org|net|gov|edu|int|mil|co|uk|de|fr|es|it|ru|su)', '', combined_string, flags=re.IGNORECASE)
    tokens = re.split(r'/', cleaned_string)
    tokens = [token for token in tokens if token]
    return tokens


def get_url_info(url):
    page_title = clean_url(url)
    page_description = ''
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            page_title = soup.title.string if soup.title else page_title
            meta_description = soup.find('meta', attrs={'name': 'description'})
            page_description = meta_description['content'] if meta_description else page_description
    except Exception as e:
        pass 

    return {
        'url': url,
        'page_title': page_title,
        'page_description': page_description
    }


In [76]:
url = "https://scikit-learn.org/stable/modules/clustering.html"
get_candidates(get_url_info(url)['page_title'])

computational-biology
bioinformatics
neo4j


In [77]:
url = "https://www.youtube.com/watch?v=8Ij7A1VCB7I&ab_channel=MarkTilbury"
get_candidates(get_url_info(url)['page_title'])

logic-pro
brave-new-world
freelancing


In [78]:
url = "https://www.manning.com/books/machine-learning-system-design"
get_candidates(get_url_info(url)['page_title'])

design-systems
ai-modern-approach
computational-biology


In [80]:
url = "https://learn-anything.xyz"
get_candidates(get_url_info(url)['page_title'])

strength-training
cracking-the-coding-interview
caddy
